In [1]:
import pickle
from pyhealth.datasets import split_by_patient
from torch_geometric.utils import to_networkx, from_networkx

with open('../../../data/pj20/exp_data/ccscm_ccsproc_atc3/sample_dataset_mimic4_mortality_th015.pkl', 'rb') as f:
    sample_dataset = pickle.load(f)

with open('../../../data/pj20/exp_data/ccscm_ccsproc_atc3/graph_mimic4_mortality_th015.pkl', 'rb') as f:
    G = pickle.load(f)

G_tg = from_networkx(G) 
train_dataset, val_dataset, test_dataset = split_by_patient(sample_dataset, [0.8, 0.1, 0.1], seed=528)

/home/pj20/miniconda3/envs/kgc/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
c_v, p_v, d_v = [], [], []

for patient in sample_dataset:
    c_v.append(len(patient['conditions']))
    p_v.append(len(patient['procedures']))
    d_v.append(len(patient['drugs']))

print(max(c_v), max(p_v), max(d_v))
max_visits = max(c_v)

1 1 1


In [3]:
# from sklearn.decomposition import PCA
# import torch

# # Define the target dimensionality
# target_dim = 512

# # Fit PCA to the word embeddings
# pca = PCA(n_components=target_dim)
# G_tg.x = torch.tensor(pca.fit_transform(G_tg.x))

In [6]:
sample_dataset[0]

{'visit_id': '22595853',
 'patient_id': '10000032',
 'conditions': [['151', '6', '127', '657', '651', '663']],
 'procedures': [['88']],
 'drugs': [['B01A',
   'J07B',
   'A12B',
   'C03D',
   'C03C',
   'N02B',
   'J05A',
   'R03A',
   'N07B',
   'R03B']],
 'label': 0,
 'node_set': [3,
  5,
  2057,
  10,
  11,
  12,
  18,
  21,
  2073,
  2074,
  2077,
  30,
  36,
  2085,
  4134,
  2086,
  4136,
  4137,
  4138,
  2090,
  2092,
  2093,
  48,
  2097,
  4144,
  4148,
  55,
  57,
  58,
  63,
  2112,
  66,
  69,
  4168,
  73,
  75,
  77,
  79,
  80,
  2129,
  82,
  88,
  89,
  90,
  2138,
  91,
  2145,
  2147,
  100,
  99,
  2151,
  105,
  2162,
  115,
  116,
  118,
  2167,
  121,
  2170,
  122,
  4220,
  127,
  129,
  130,
  133,
  4230,
  135,
  2184,
  2187,
  139,
  2189,
  144,
  2193,
  146,
  2198,
  2201,
  154,
  153,
  2205,
  160,
  163,
  167,
  168,
  169,
  171,
  172,
  175,
  176,
  177,
  181,
  185,
  188,
  189,
  2236,
  2241,
  194,
  195,
  2249,
  2251,
  2253,
  209,


In [4]:
import torch
from torch_geometric.loader import DataLoader



def get_subgraph(G, dataset, idx):
    patient = dataset[idx]
    L = G.edge_subgraph(torch.tensor(patient['node_set']))
    P = L.subgraph(torch.tensor(patient['node_set']))
    P.label = patient['label']
    P.visits_cond = patient['visit_node_set_condition']
    P.visits_proc = patient['visit_node_set_procedure']
    P.visits_drug = patient['visit_node_set_drug']
    
    return P

class Dataset(torch.utils.data.Dataset):
    def __init__(self, G, dataset):
        self.G = G
        self.dataset=dataset
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        return get_subgraph(G=self.G, dataset=self.dataset, idx=idx)


In [5]:
P = get_subgraph(G_tg, train_dataset, 1)

KeyError: 'visit_node_set_condition'

In [ ]:
len(P.visits_cond[0])

4599

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GATConv, GINConv, HGTConv
from torch_geometric.data import DataLoader, Data
from torch_geometric.nn import global_mean_pool, global_max_pool
from torch_geometric.nn import DataParallel
from torch_geometric.loader import DataListLoader

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads)
        self.conv2 = GATConv(hidden_channels*heads, hidden_channels, heads=heads)
        self.conv3 = GATConv(hidden_channels*heads, hidden_channels, heads=1)

        self.fc = Linear(hidden_channels, out_channels)
        
    def forward(self, x, edge_index, batch):
        x = F.elu(self.conv1(x, edge_index))
        # print(x.shape)
        x = F.elu(self.conv2(x, edge_index))
        # print(x.shape)
        x = F.elu(self.conv3(x, edge_index))
        # print(x.shape)
        x = global_mean_pool(x, batch)
        # print(x.shape)
        x = F.dropout(x, p=0.5, training=self.training)
        # print(x.shape)
        logits = self.fc(x)
        # print(logits.shape)
        return logits


class GIN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GIN, self).__init__()
        self.conv1 = GINConv(Linear(in_channels, hidden_channels))
        self.conv2 = GINConv(Linear(hidden_channels, hidden_channels))
        self.conv3 = GINConv(Linear(hidden_channels, hidden_channels))

        self.fc = Linear(hidden_channels, out_channels)
        
    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        x = global_mean_pool(x, batch)
        x = F.dropout(x, p=0.5, training=self.training)
        logits = self.fc(x)
        return logits


class GINX(torch.nn.Module):
    def __init__(self, num_nodes, embedding_dim, hidden_channels, out_channels, word_emb=None):
        super(GINX, self).__init__()
        
        if word_emb == None:
            self.embedding = torch.nn.Embedding(num_nodes, embedding_dim)
            self.conv1 = GINConv(Linear(embedding_dim, hidden_channels))
        else:
            self.embedding = torch.nn.Embedding.from_pretrained(word_emb, freeze=False)
            self.conv1 = GINConv(Linear(word_emb.shape[1], hidden_channels))

        self.conv2 = GINConv(Linear(hidden_channels, hidden_channels))
        self.conv3 = GINConv(Linear(hidden_channels, hidden_channels))
        self.fc = Linear(hidden_channels, out_channels)
        
    def forward(self, node_ids, edge_index, batch):
        x = self.embedding(node_ids)
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        x = global_mean_pool(x, batch)
        x = F.dropout(x, p=0.5, training=self.training)
        logits = self.fc(x)
        return logits


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GINEConv
from pyhealth.models import RETAINLayer

class GraphCare(nn.Module):
    def __init__(self, num_nodes, feature_keys, embedding_dim, hidden_dim, out_channels, dropout=0.5, max_visits=None, word_emb=None, use_attn=True):
        super(GraphCare, self).__init__()
        self.max_visits = max_visits
        self.max_nodes = len(word_emb)
        self.embedding_dim = embedding_dim
        self.use_attn = use_attn
        self.alpha = nn.Parameter(torch.tensor(0.5))

        if word_emb == None:
            self.embedding = torch.nn.Embedding(num_nodes, embedding_dim)
        else:
            self.embedding = torch.nn.Embedding.from_pretrained(word_emb, freeze=False)
        
        self.retain = nn.ModuleDict()
        for feature_key in feature_keys:
            self.retain[feature_key] = RETAINLayer(feature_size=self.max_nodes, dropout=dropout)
        
        self.conv1 = GINEConv(nn.Linear(embedding_dim, hidden_dim), edge_dim=1)
        self.conv2 = GINEConv(nn.Linear(hidden_dim, hidden_dim), edge_dim=1)
        self.conv3 = GINEConv(nn.Linear(hidden_dim, hidden_dim), edge_dim=1)

        self.fc = nn.Linear(hidden_dim, out_channels)

    def forward(self, node_ids, edge_index, batch, visits_cond, visits_proc, visits_drug):
        x = self.embedding(node_ids)  # (num_nodes, embedding_dim)
        if self.use_attn == True:
            cond_attn = self.retain['cond'](visits_cond)
            proc_attn = self.retain['proc'](visits_proc)
            drug_attn = self.retain['drug'](visits_drug)
            cross_attn = self.retain['cross'](visits_cond + visits_proc + visits_drug)

            attn = (cond_attn.add_(proc_attn).add_(drug_attn).add_(cross_attn))   # (batch_size, max_nodes)
            
            # Create a batch index tensor to map the batch index to the corresponding attention weight
            batch_index = torch.arange(attn.size(0), device=node_ids.device).repeat_interleave(torch.bincount(batch))   
            # print("batch index shape: ", batch_index.shape)
            # print("edge index shape: ", edge_index.shape)
            # Fill the attn_weights matrix with the correct weights using batch_index and node_ids
            attn_weights = attn[batch_index, node_ids]
            # Multiply the embeddings with the corresponding attention weights
            # x = x * attn_weights
            # x = (1 - self.alpha) * x + self.alpha * x
            row, col = edge_index
            # Define a small constant value epsilon
            epsilon = 1e-6

            # Calculate the geometric mean with added epsilon
            # Normalize the attn_weights and replace NaNs with 0s
            attn_weights = attn_weights / torch.max(attn_weights)
            attn_weights = torch.where(torch.isnan(attn_weights), torch.zeros_like(attn_weights), attn_weights)

            # Calculate the geometric mean with added epsilon
            edge_attr = ((attn_weights[row] + epsilon) + (attn_weights[col] + epsilon)).unsqueeze(-1)

        # Apply the first GIN layer
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        # Apply the second GIN layer
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        x = F.relu(self.conv3(x, edge_index, edge_attr))

        x = global_mean_pool(x, batch)
        x = F.dropout(x, p=0.5, training=self.training)

        logits = self.fc(x)
        return logits



In [ ]:
from tqdm import tqdm
from pyhealth.metrics import multilabel_metrics_fn
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, jaccard_score
    
def train(model, device, train_loader, optimizer, model_):
    model.train()
    training_loss = 0
    tot_loss = 0
    pbar= tqdm(enumerate(train_loader))
    for i, data in pbar:
        pbar.set_description(f'loss: {training_loss}')

        data = data.to(device)
        optimizer.zero_grad()
        if model_ == "GIN":
            out = model(data.x, data.edge_index, data.batch)
        elif model_ == "GINX":
            out = model(data.y, data.edge_index, data.batch)
        else:
            out = model(
                    data.y, 
                    data.edge_index, 
                    data.batch, 
                    data.visits_cond.reshape(int(train_loader.batch_size), int(len(data.visits_cond)/train_loader.batch_size), data.visits_cond.shape[1]).double(), 
                    data.visits_proc.reshape(int(train_loader.batch_size), int(len(data.visits_proc)/train_loader.batch_size), data.visits_proc.shape[1]).double(), 
                    data.visits_drug.reshape(int(train_loader.batch_size), int(len(data.visits_drug)/train_loader.batch_size), data.visits_drug.shape[1]).double(),
                )
        try:
            label = data.label.reshape(int(train_loader.batch_size), int(len(data.label)/train_loader.batch_size))
        except:
            continue

        loss = F.binary_cross_entropy_with_logits(out, label.float())
        loss.backward()
        training_loss = loss
        tot_loss += loss
        optimizer.step()
    
    return tot_loss

def evaluate(model, device, loader, model_):
    model.eval()
    y_prob_all = []
    y_true_all = []

    for data in tqdm(loader):
        data = data.to(device)
        with torch.no_grad():    
            
            if model_ == "GIN":
                logits = model(data.x, data.edge_index, data.batch)
            elif model_ == "GINX":
                logits = model(data.y, data.edge_index, data.batch)
            else:
                logits = model(
                    data.y, 
                    data.edge_index, 
                    data.batch, 
                    data.visits_cond.reshape(int(loader.batch_size), int(len(data.visits_cond)/loader.batch_size), data.visits_cond.shape[1]).double(), 
                    data.visits_proc.reshape(int(loader.batch_size), int(len(data.visits_proc)/loader.batch_size), data.visits_proc.shape[1]).double(), 
                    data.visits_drug.reshape(int(loader.batch_size), int(len(data.visits_drug)/loader.batch_size), data.visits_drug.shape[1]).double(),
                )

            y_prob = torch.sigmoid(logits)
            try:
                y_true = data.label.reshape(int(loader.batch_size), int(len(data.label)/loader.batch_size))
            except:
                continue
            y_prob_all.append(y_prob.cpu())
            y_true_all.append(y_true.cpu())
            
    y_true_all = np.concatenate(y_true_all, axis=0)
    y_prob_all = np.concatenate(y_prob_all, axis=0)

    return y_true_all, y_prob_all

def train_loop(train_loader, val_loader, model, optimizer, device, epochs, model_):
    best_acc = 0
    best_f1 = 0
    for epoch in range(1, epochs+1):
        loss = train(model, device, train_loader, optimizer, model_)
        y_true_all, y_prob_all = evaluate(model, device, val_loader, model_)

        y_pred_all = (y_prob_all >= 0.5).astype(int)
        
        val_pr_auc = average_precision_score(y_true_all, y_prob_all)
        val_roc_auc = roc_auc_score(y_true_all, y_prob_all)
        val_jaccard = jaccard_score(y_true_all, y_pred_all, average='macro', zero_division=1)
        val_acc = accuracy_score(y_true_all, y_pred_all)
        val_f1 = f1_score(y_true_all, y_pred_all, average='macro', zero_division=1)
        val_precision = precision_score(y_true_all, y_pred_all, average='macro', zero_division=1)
        val_recall = recall_score(y_true_all, y_pred_all, average='macro', zero_division=1)

        if val_acc >= best_acc and val_f1 >= best_f1:
            torch.save(model.state_dict(), '../../../data/pj20/exp_data/saved_weights_gin_mimic3_readmission_dynamic.pkl')
            print("best model saved")
            best_acc = val_acc
            best_f1 = val_f1

        print(f'Epoch: {epoch}, Training loss: {loss}, Val PRAUC: {val_pr_auc:.4f}, Val ROC_AUC: {val_roc_auc:.4f}, Val acc: {val_acc:.4f}, Val F1: {val_f1:.4f}, Val precision: {val_precision:.4f}, Val recall: {val_recall:.4f}, Val jaccard: {val_jaccard:.4f}')


In [ ]:
train_set = Dataset(G=G_tg, dataset=train_dataset)
val_set = Dataset(G=G_tg, dataset=val_dataset)
test_set = Dataset(G=G_tg, dataset=test_dataset)

train_loader = DataLoader(train_set, batch_size=16, shuffle=True, drop_last=True)
val_loader = DataLoader(val_set, batch_size=16, shuffle=False, drop_last=True)
test_loader = DataLoader(test_set, batch_size=16, shuffle=False, drop_last=True)

/home/pj20/miniconda3/envs/kgc/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
import numpy as np
for i, data in enumerate(train_loader):
    # batch_size = torch.max(data.batch).item() + 1  # number of batches
    # batch_x = []
    # for i in range(batch_size):
    #     batch_mask = (data.batch == i)
    #     batch_nodes = torch.sum(batch_mask).item()  # number of nodes in this batch
    #     batch_x.append(torch.masked_select(data.y, batch_mask).view(batch_nodes, -1))
    #     print(len(batch_x[i]))
    print(data.visits_cond.shape)
    break

torch.Size([16, 4599])


In [ ]:
model_ = "GINX"

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

if model_ == "GIN":
    in_channels = train_set[0].x.shape[1]
    model = GIN(in_channels=in_channels, out_channels=1, hidden_channels=512).to(device)
    # model = GAT(in_channels=in_channels, out_channels=1, hidden_channels=256, heads=3).to(device)
    # model = HGT(in_channels=in_channels, out_channels=out_channels, hidden_channels=512, heads=2).to(device)
elif model_ == "GINX":
    model = GINX(num_nodes=G_tg.num_nodes, embedding_dim=512, hidden_channels=512, out_channels=1, word_emb=G_tg.x).to(device)

elif model_ == "GraphCare":
    # model = GINX(num_nodes=G_tg.num_nodes, embedding_dim=512, hidden_channels=512, out_channels=out_channels, word_emb=G_tg.x).to(device)
    model = GraphCare(
        num_nodes=G_tg.num_nodes,
        feature_keys=['cond', 'proc', 'cross'], 
        embedding_dim=len(G_tg.x[0]), 
        hidden_dim=512, 
        out_channels=1, 
        dropout=0.5, 
        max_visits=max_visits,
        word_emb=G_tg.x,
        use_attn=True
    ).to(device)

model.double()

GINX(
  (embedding): Embedding(4599, 1536)
  (conv1): GINConv(nn=Linear(in_features=1536, out_features=512, bias=True))
  (conv2): GINConv(nn=Linear(in_features=512, out_features=512, bias=True))
  (conv3): GINConv(nn=Linear(in_features=512, out_features=512, bias=True))
  (fc): Linear(in_features=512, out_features=1, bias=True)
)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_loop(train_loader=train_loader, val_loader=val_loader, model=model, optimizer=optimizer, device=device, epochs=100, model_=model_)

In [ ]:
# torch.save(model.state_dict(), './exp_data/saved_weights_gat_mimic3_drugrec.pkl')
# torch.save(model.state_dict(), './exp_data/saved_weights_gin_mimic3_readmission.pkl')
# torch.save(model.state_dict(), './exp_data/saved_weights_hgt_mimic3_drugrec.pkl')
torch.save(model.state_dict(), '../../../data/pj20/exp_data/saved_weights_graphcare_mimic3_mortality_th02.pkl')

In [ ]:
# model.load_state_dict(torch.load('./exp_data/saved_weights_gat_mimic3_drugrec.pkl'))
# model.load_state_dict(torch.load('./exp_data/saved_weights_gin_mimic3_readmission_dynamic.pkl'))
# model.double()

y_true_all, y_prob_all = evaluate(model, device, val_loader, static)

y_pred_all = (y_prob_all >= 0.5).astype(int)

test_pr_auc = average_precision_score(y_true_all, y_prob_all)
test_roc_auc = roc_auc_score(y_true_all, y_prob_all)
test_jaccard = jaccard_score(y_true_all, y_pred_all, average='macro', zero_division=1)
test_acc = accuracy_score(y_true_all, y_pred_all)
test_f1 = f1_score(y_true_all, y_pred_all, average='macro', zero_division=1)
test_precision = precision_score(y_true_all, y_pred_all, average='macro', zero_division=1)
test_recall = recall_score(y_true_all, y_pred_all, average='macro', zero_division=1)

print(f'test PRAUC: {test_pr_auc:.4f}, test ROC_AUC: {test_roc_auc:.4f}, test acc: {test_acc:.4f}, test F1: {test_f1:.4f}, test precision: {test_precision:.4f}, test recall: {test_recall:.4f}, test jaccard: {test_jaccard:.4f}')


100%|██████████| 61/61 [00:29<00:00,  2.07it/s]

test PRAUC: 0.0854, test ROC_AUC: 0.5622, test acc: 0.9232, test F1: 0.4800, test precision: 0.9616, test recall: 0.5000, test jaccard: 0.4616


In [ ]:
from sparknlp.pretrained import PretrainedPipeline 
pipeline = PretrainedPipeline( "icd10cm_umls_mapping","en","clinical/models")
pipeline.annotate(["M8950", "R822", "R0901"])